In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy as sp
import numpy as np
from scipy import odr


import datetime
from datetime import timedelta
from datetime import datetime
from datetime import date
from dateutil.relativedelta import *
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 300)

#RETH BEI SEEFELD
#input start run time
start = pd.to_datetime('2022-01-01 1:00')
end   = pd.to_datetime('2022-12-31 23:00')


#Location. Lat and Long and Timezone

location = (47.299, 11.204)
timezone=+2

#Mountains or big trees above the horizon
eastmountainangle=35
westmountainangle=15


#side toward Auland #photaz=330
#side toward Leiten #photaz=150
#House Facing #photaz=240
#Due South #photaz=180

#Panel orientation elevation where zero is lying flat and zero is pointing north
photangle=40
photaz=150

#Number of panels
panelcount=16

#Percent loss due to clouds
cloudloss=0.45
cloudwin=1-cloudloss


#Losses in the solar cells
panelefficiency=0.18


#panel size
panellength=2
panellwidth=1




In [22]:
#HAMILTON GA
#input start run time
start = pd.to_datetime('2022-07-03 12:00')
end   = pd.to_datetime('2022-07-01 23:00')


#Location. Lat and Long and Timezone
location = (33, -84)


timezone=-4

#Mountains or big trees above the horizon
eastmountainangle=20
westmountainangle=35


#Chances Barn is West at 270 degrees?
#Panel orientation elevation where zero is lying flat and zero is pointing north
photangle=15
photaz=270

#Number of panels
panelcount=48

#Percent loss due to clouds
cloudloss=0.45
cloudwin=1-cloudloss


#Losses in the solar cells
panelefficiency=0.17


#panel size
panellength=2
panellwidth=1


In [23]:
#find totalarea in m^2
panelarea=panellength*panellwidth*panelcount


#Number of hours between start and end
timediff=end-start


#point in the sky that the photovoltaic points at
photel=90-photangle


In [24]:


#Function for positive sun angle. This is respective to the angle of the panels
def OnlyPosAng(num):
    if 180>num>=0: return num
    if num<0: return 0
    if num>=180: return 0


#Make a function that makes all negative numbers 0. It is used for power
def OnlyPosNum(num):
    if num>=0: return num
    if num<0: return 0


def OverMountains(azimuth, elevation):
 if azimuth<180:
    if elevation>eastmountainangle:
        return 1
    if elevation<=eastmountainangle:
        return 0
 if azimuth>180:
    if elevation>westmountainangle:
        return 1
    if elevation<=westmountainangle:
        return 0



In [25]:
#timediff counts the number of hours between start and finnish
timediffhours=24*timediff.days+timediff.seconds/3600

In [26]:
# This  function returns sun position and power based on location and time

def sunpos(when, location):
# Extract the passed data
    year, month, day, hour, minute, second= when
    latitude, longitude = location
# Math typing shortcuts
    rad, deg = math.radians, math.degrees
    sin, cos, tan = math.sin, math.cos, math.tan
    asin, atan2 = math.asin, math.atan2
# Convert latitude and longitude to radians
    rlat = rad(latitude)
    rlon = rad(longitude)
# Decimal hour of the day at Greenwich
    greenwichtime = hour - timezone + minute / 60 + second / 3600
# Days from J2000, accurate from 1901 to 2099
    daynum = (
        367 * year
        - 7 * (year + (month + 9) // 12) // 4
        + 275 * month // 9
        + day
        - 730531.5
        + greenwichtime / 24
    )
# Mean longitude of the sun
    mean_long = daynum * 0.01720279239 + 4.894967873
# Mean anomaly of the Sun
    mean_anom = daynum * 0.01720197034 + 6.240040768
# Ecliptic longitude of the sun
    eclip_long = (
        mean_long
        + 0.03342305518 * sin(mean_anom)
        + 0.0003490658504 * sin(2 * mean_anom)
    )
# Obliquity of the ecliptic
    obliquity = 0.4090877234 - 0.000000006981317008 * daynum
# Right ascension of the sun
    rasc = atan2(cos(obliquity) * sin(eclip_long), cos(eclip_long))
# Declination of the sun
    decl = asin(sin(obliquity) * sin(eclip_long))
# Local sidereal time
    sidereal = 4.894961213 + 6.300388099 * daynum + rlon
# Hour angle of the sun
    hour_ang = sidereal - rasc
# Local elevation of the sun
    elevation = asin(sin(decl) * sin(rlat) + cos(decl) * cos(rlat) * cos(hour_ang))
# Local azimuth of the sun
    azimuth = atan2(
        -cos(decl) * cos(rlat) * sin(hour_ang),
        sin(decl) - sin(rlat) * sin(elevation),
    )
# Convert azimuth and elevation to degrees
    azimuth = into_range(deg(azimuth), 0, 360)
    elevation = into_range(deg(elevation), -180, 180)
    

        
        
#Data table for Elevation to Power. This Data was found online
    data = {'Sun_Elevation': [90, 75, 60, 45, 30, 20, 15, 10, 5, 0], 
        'Power': [1040, 1030, 1000, 950, 840, 710, 620, 470, 260, 20]}
    df_ElevationPower = pd.DataFrame(data)

    poly_model = odr.polynomial(5)
    data = odr.Data(df_ElevationPower.Sun_Elevation, df_ElevationPower.Power)
    odr_obj = odr.ODR(data, poly_model)
    output = odr_obj.run() 
    poly = np.poly1d(output.beta[::-1])
    poly_y = poly(df_ElevationPower.Sun_Elevation)
    df_gen_Elevation = pd.DataFrame({'gen':range(101)})
    gen_y=poly(df_gen_Elevation)
    

    
    power= OnlyPosNum(round(OverMountains(azimuth, elevation)*poly(elevation),1))    
   
 # Return azimuth and elevation in degrees
    return (round(azimuth, 2), round(elevation, 2), round(power,2))
def into_range(x, range_min, range_max):
    shiftedx = x - range_min
    delta = range_max - range_min
    return (((shiftedx % delta) + delta) % delta) + range_min



In [27]:
#plt.plot(df_ElevationPower.Sun_Elevation,  df_ElevationPower.Power,'ro', label="input data")
#plt.plot(df_gen_Elevation, gen_y, label="polynomial ODR")

In [28]:

#Convert the input data from above to more readable variables
start_year= start.year
start_month= start.month
start_day=start.day
start_hour=start.hour
start_minute=start.minute
start_second=start.second

#year, month, day, hour, minute, secondfrom time start input above
when=(start_year,start_month,start_day,start_hour, start_minute, 0)


#The function
sun_posdata=sunpos(when, location)


#convert the "when" tuple to data frame data and unpacks the tuple into datetime arguments
dt_date = datetime(*when) 

#Use the sun position data and the photovoltaic panel position to find the angle between the two and the incident ratio of the sun´s power onto the panel
sunaz=    sun_posdata[0]
sunel=    sun_posdata[1]
sunpower= sun_posdata[2]

incidence_angle=OnlyPosAng(round(180/math.pi*math.acos(math.sin(sunel/180*math.pi)*math.sin(photel/180*math.pi)+math.cos(sunel/180*math.pi)*math.cos(photel/180*math.pi)*math.cos(sunaz/180*math.pi-photaz
 /180*math.pi)),0))

#Find the incident ratio according to angle of the panel to the sun
incidence_ratio=math.cos(incidence_angle/180*math.pi)

#Estimate the amount of power hitting the panel at any given time considering sun position, sun power, panel area, panel position and cloud cover
incident_power=incidence_ratio*sunpower*panelarea*panelefficiency*cloudwin


df_data=pd.DataFrame({'Date_Time':[dt_date],'Sun_Azimuth':[sunaz],'Sun_Elevation':[sunel],'Sun_Power':[sunpower],'Angle_SuntoPanel':[incidence_angle],'PowerRatio_SuntoPanel':[incidence_ratio], 'Incident_Power':[incident_power]})



#Before the loop begins cont is the start formatted date. l is the start accumulated number
cont=start

l=0
total_incident_power=incident_power


#For finding the total incident power for each month, these if statements set all months that are not in the time range to zero.  
#For those that are within the range, the zero will be assigned to the accumulated power once the iteration comes to that point

if start.month==1: January_incident_power=incident_power
else: January_incident_power=0

if start.month==2: February_incident_power=incident_power
else: February_incident_power=0

if start.month==3: March_incident_power=incident_power
else: March_incident_power=0

if start.month==4: April_incident_power=incident_power
else: April_incident_power=0

if start.month==5: May_incident_power=incident_power
else: May_incident_power=0    

if start.month==6: June_incident_power=incident_power
else: June_incident_power=0 
    
if start.month==7: July_incident_power=incident_power
else: July_incident_power=0 

if start.month==8: August_incident_power=incident_power
else: August_incident_power=0 

if start.month==9: September_incident_power=incident_power
else: September_incident_power=0 
    
if start.month==10: October_incident_power=incident_power
else: October_incident_power=0
    
if start.month==11: November_incident_power=incident_power
else: November_incident_power=0

if start.month==12: December_incident_power=incident_power
else: December_incident_power=0
    
    
    
while l<timediffhours:  


 cont=cont+ timedelta(hours=1)
 cont_year= cont.year
 cont_month= cont.month
 cont_day=cont.day
 cont_hour=cont.hour
 cont_minute=cont.minute
 cont_second=cont.second

 when=(cont_year,cont_month,cont_day,cont_hour, cont_minute, 0)
 sun_posdata=sunpos(when, location)

 my_tuple = when
 dt_date = datetime(*my_tuple) #Unpacks the tuple into datetime arguments

    
 #use the sun position data and the photovoltaic panel position to find the angle between the two and the incident ratio of the sun´s power onto the panel
 sunaz=    sun_posdata[0]
 sunel=    sun_posdata[1]
 sunpower= sun_posdata[2]

 incidence_angle=OnlyPosAng(round(180/math.pi*math.acos(math.sin(sunel/180*math.pi)*math.sin(photel/180*math.pi)+math.cos(sunel/180*math.pi)*math.cos(photel/180*math.pi)*math.cos(sunaz/180*math.pi-photaz
 /180*math.pi)),0))


#These values change with every iteration of the loop. They have to be updated

 incidence_ratio=math.cos(incidence_angle/180*math.pi)
 incident_power=OnlyPosNum(incidence_ratio*sunpower*panelarea*panelefficiency*cloudwin)
 total_incident_power=total_incident_power+incident_power


 if  dt_date.month==1:    
    January_incident_power= January_incident_power+incident_power      
    
 if  dt_date.month==2:    
    February_incident_power= February_incident_power+incident_power        
    
 if  dt_date.month==3:    
    March_incident_power= March_incident_power+incident_power
        
 if  dt_date.month==4:    
    April_incident_power= April_incident_power+incident_power
    
 if  dt_date.month==5:    
    May_incident_power= May_incident_power+incident_power
    
 if  dt_date.month==6:    
    June_incident_power= June_incident_power+incident_power
    
 if  dt_date.month==7:    
    July_incident_power= July_incident_power+incident_power
    
 if  dt_date.month==8:    
    August_incident_power= August_incident_power+incident_power

 if  dt_date.month==9:    
    September_incident_power= September_incident_power+incident_power
    
 if  dt_date.month==10:    
    October_incident_power= October_incident_power+incident_power

 if  dt_date.month==11:    
    November_incident_power= November_incident_power+incident_power
    
 if  dt_date.month==12:    
    December_incident_power= December_incident_power+incident_power

 l=l+1

    

#make the date format shorter
i=len(df_data)
q=0
while i>0:         
    df_data.at[q, 'Date_Time'] =df_data.at[q,'Date_Time'].strftime(" %a %d-%m %H:%M")
    q=q+1
    i=i-1

#Display a one liner of inputs and outputs for the beginning time. This is good for getting the actual power at any time
df_data

,Date_Time,Sun_Azimuth,Sun_Elevation,Sun_Power,Angle_SuntoPanel,PowerRatio_SuntoPanel,Incident_Power
0,Sun 03-07 12:00,108.12,65.75,1014.2,39.0,0.777146,7074.716553


In [29]:
print("January   ", round(January_incident_power/1000))
print("February  ", round(February_incident_power/1000))
print("March     ", round(March_incident_power/1000))
print("April     ", round(April_incident_power/1000))
print("May       ", round(May_incident_power/1000))
print("June      ", round(June_incident_power/1000))
print("July      ", round(July_incident_power/1000))
print("August    ", round(August_incident_power/1000))
print("September ", round(September_incident_power/1000))
print("October   ", round(October_incident_power/1000))
print("November  ", round(November_incident_power/1000))
print("December  ", round(December_incident_power/1000))

print("          Total kWh    ",round(total_incident_power/1000))

January    0
February   0
March      0
April      0
May        0
June       0
July       7
August     0
September  0
October    0
November   0
December   0
          Total kWh     7
